In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T


spark=(
    SparkSession.builder.
    appName("Recipes ML Model - Are you a dessert?").
    config("spark.driver.memory","8g").
    getOrCreate()
)

spark

In [2]:
food=spark.read.csv("epi_r.csv",inferSchema=True, header=True)

In [3]:
food.printSchema()

root
 |-- title: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- calories: string (nullable = true)
 |-- protein: double (nullable = true)
 |-- fat: double (nullable = true)
 |-- sodium: double (nullable = true)
 |-- #cakeweek: double (nullable = true)
 |-- #wasteless: double (nullable = true)
 |-- 22-minute meals: double (nullable = true)
 |-- 3-ingredient recipes: double (nullable = true)
 |-- 30 days of groceries: double (nullable = true)
 |-- advance prep required: double (nullable = true)
 |-- alabama: double (nullable = true)
 |-- alaska: double (nullable = true)
 |-- alcoholic: double (nullable = true)
 |-- almond: double (nullable = true)
 |-- amaretto: double (nullable = true)
 |-- anchovy: double (nullable = true)
 |-- anise: double (nullable = true)
 |-- anniversary: double (nullable = true)
 |-- anthony bourdain: double (nullable = true)
 |-- aperitif: double (nullable = true)
 |-- appetizer: double (nullable = true)
 |-- apple: double (nullable = true)


In [4]:
food.count()

20057

In [5]:
len(food.columns)

680

In [6]:
def sanitize_coloumn_name(name):

    answer = name
    
    for i,j in (
        (" ","_",),
        ("-","_",),
        ("/","_",),
        ("&","and",),
    ):
        answer = answer.replace(i,j)

    return "".join(
        [char for char in answer if char.isalpha() or char.isdigit() or char == "_"]
    )

food=food.toDF(*[sanitize_coloumn_name(name) for name in food.columns])



In [7]:
food.printSchema()

root
 |-- title: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- calories: string (nullable = true)
 |-- protein: double (nullable = true)
 |-- fat: double (nullable = true)
 |-- sodium: double (nullable = true)
 |-- cakeweek: double (nullable = true)
 |-- wasteless: double (nullable = true)
 |-- 22_minute_meals: double (nullable = true)
 |-- 3_ingredient_recipes: double (nullable = true)
 |-- 30_days_of_groceries: double (nullable = true)
 |-- advance_prep_required: double (nullable = true)
 |-- alabama: double (nullable = true)
 |-- alaska: double (nullable = true)
 |-- alcoholic: double (nullable = true)
 |-- almond: double (nullable = true)
 |-- amaretto: double (nullable = true)
 |-- anchovy: double (nullable = true)
 |-- anise: double (nullable = true)
 |-- anniversary: double (nullable = true)
 |-- anthony_bourdain: double (nullable = true)
 |-- aperitif: double (nullable = true)
 |-- appetizer: double (nullable = true)
 |-- apple: double (nullable = true)
 |

In [8]:
#for x in food.columns:
#    food.select(x).summary().show()

In [9]:
food.select(F.size(F.collect_set('turkey')) == 2).show()

+-------------------------------+
|(size(collect_set(turkey)) = 2)|
+-------------------------------+
|                           true|
+-------------------------------+



In [10]:
binary=food.agg(*[(F.size(F.collect_set(x)) == 2).alias(x) for x in food.columns])

In [11]:
binary.select('title','turkey').show()

+-----+------+
|title|turkey|
+-----+------+
|false|  true|
+-----+------+



In [12]:
binary=binary.toPandas()

In [13]:
import pandas as pd
pd.set_option("display.max_rows",1000)

In [14]:
binary.unstack()

title                     0    False
rating                    0    False
calories                  0    False
protein                   0    False
fat                       0    False
sodium                    0    False
cakeweek                  0    False
wasteless                 0    False
22_minute_meals           0     True
3_ingredient_recipes      0     True
30_days_of_groceries      0     True
advance_prep_required     0     True
alabama                   0     True
alaska                    0     True
alcoholic                 0     True
almond                    0     True
amaretto                  0     True
anchovy                   0     True
anise                     0     True
anniversary               0     True
anthony_bourdain          0     True
aperitif                  0     True
appetizer                 0     True
apple                     0     True
apple_juice               0     True
apricot                   0     True
arizona                   0     True
a

In [15]:
binary=food.agg(*[(F.size(F.collect_set(x)) == 2).alias(x) for x in food.columns])

In [16]:
food.select(F.collect_set('cakeweek')).show(truncate=False)

+-------------------------------+
|collect_set(cakeweek)          |
+-------------------------------+
|[0.0, 1.0, 1188.0, 24.0, 880.0]|
+-------------------------------+



In [17]:
food.agg(*[F.collect_set(x) for x in ('cakeweek','wasteless')]).show(1,False)

+-------------------------------+----------------------+
|collect_set(cakeweek)          |collect_set(wasteless)|
+-------------------------------+----------------------+
|[0.0, 1.0, 1188.0, 24.0, 880.0]|[0.0, 1.0, 1439.0]    |
+-------------------------------+----------------------+



In [18]:
(
    food.where("cakeweek > 1.0 or wasteless > 1.0").
    select("title", "rating","wasteless","cakeweek", food.columns[-2]).
    show(truncate=False)
)

+----------------------------------------------------------------+------------------------+---------+--------+----------+
|title                                                           |rating                  |wasteless|cakeweek|snack_week|
+----------------------------------------------------------------+------------------------+---------+--------+----------+
|"Beet Ravioli with Pine Nut ""Goat Cheese"" Rosemary-Cream Sauce| Aged Balsamic Vinegar "|0.0      |880.0   |0.0       |
|"Seafood ""Cataplana"" with Saffron                             | Vermouth               |1439.0   |24.0    |0.0       |
|"""Pot Roast"" of Seitan                                        | Aunt Gloria-Style "    |0.0      |1188.0  |0.0       |
+----------------------------------------------------------------+------------------------+---------+--------+----------+



In [19]:
food.select(F.col("cakeweek").isin([0.0,1.0])).show()

+------------------------+
|(cakeweek IN (0.0, 1.0))|
+------------------------+
|                    true|
|                    true|
|                    true|
|                    true|
|                    true|
|                    true|
|                    true|
|                    true|
|                    true|
|                    true|
|                    true|
|                    true|
|                    true|
|                    true|
|                    true|
|                    true|
|                    true|
|                    true|
|                    true|
|                    true|
+------------------------+
only showing top 20 rows



In [20]:
food = (
    food.where(
        (F.col("cakeweek").isin([0.0,1.0]) | F.col("cakeweek").isNull()) &
        (F.col("wasteless").isin([0.0,1.0]) | F.col("wasteless").isNull())
    )
)

In [21]:
20057

20057

In [22]:
food.count()

20054

In [23]:
IDENTIFIERS = ["title"]

TARGET = ["dessert"]

CONTINUOUS = ["rating","calories","protein","fat","sodium"]

BINARY = [x for x in food.columns if x not in IDENTIFIERS and x not in TARGET and x not in CONTINUOUS]

BINARY

['cakeweek',
 'wasteless',
 '22_minute_meals',
 '3_ingredient_recipes',
 '30_days_of_groceries',
 'advance_prep_required',
 'alabama',
 'alaska',
 'alcoholic',
 'almond',
 'amaretto',
 'anchovy',
 'anise',
 'anniversary',
 'anthony_bourdain',
 'aperitif',
 'appetizer',
 'apple',
 'apple_juice',
 'apricot',
 'arizona',
 'artichoke',
 'arugula',
 'asian_pear',
 'asparagus',
 'aspen',
 'atlanta',
 'australia',
 'avocado',
 'back_to_school',
 'backyard_bbq',
 'bacon',
 'bake',
 'banana',
 'barley',
 'basil',
 'bass',
 'bastille_day',
 'bean',
 'beef',
 'beef_rib',
 'beef_shank',
 'beef_tenderloin',
 'beer',
 'beet',
 'bell_pepper',
 'berry',
 'beverly_hills',
 'birthday',
 'biscuit',
 'bitters',
 'blackberry',
 'blender',
 'blue_cheese',
 'blueberry',
 'boil',
 'bok_choy',
 'bon_appétit',
 'bon_apptit',
 'boston',
 'bourbon',
 'braise',
 'bran',
 'brandy',
 'bread',
 'breadcrumbs',
 'breakfast',
 'brie',
 'brine',
 'brisket',
 'broccoli',
 'broccoli_rabe',
 'broil',
 'brooklyn',
 'brown_ri

In [24]:
food = food.dropna(how="all",subset=[x for x in food.columns if x not in IDENTIFIERS])

In [25]:
20054

20054

In [26]:
food = food.dropna(subset=TARGET)

In [27]:
food.count()

20049

In [28]:
food = food.fillna(value=0.0, subset=BINARY)

In [29]:
food.where(F.col(BINARY[0]).isNull()).count()

0

In [30]:
food.printSchema()

root
 |-- title: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- calories: string (nullable = true)
 |-- protein: double (nullable = true)
 |-- fat: double (nullable = true)
 |-- sodium: double (nullable = true)
 |-- cakeweek: double (nullable = false)
 |-- wasteless: double (nullable = false)
 |-- 22_minute_meals: double (nullable = false)
 |-- 3_ingredient_recipes: double (nullable = false)
 |-- 30_days_of_groceries: double (nullable = false)
 |-- advance_prep_required: double (nullable = false)
 |-- alabama: double (nullable = false)
 |-- alaska: double (nullable = false)
 |-- alcoholic: double (nullable = false)
 |-- almond: double (nullable = false)
 |-- amaretto: double (nullable = false)
 |-- anchovy: double (nullable = false)
 |-- anise: double (nullable = false)
 |-- anniversary: double (nullable = false)
 |-- anthony_bourdain: double (nullable = false)
 |-- aperitif: double (nullable = false)
 |-- appetizer: double (nullable = false)
 |-- apple: double (nu

esta seleccionando donde rating no sea numero

In [31]:
from typing import Optional

#esto lo registra como udf
@F.udf(T.BooleanType())
def is_a_number(value: Optional[str]) -> bool:

    if not value:
        return True

    try:
        _ = float(value)
        
    except ValueError:
        return False

    return True

food.where(~is_a_number(F.col("rating"))).select(*CONTINUOUS).show()

+---------+------------+-------+----+------+
|   rating|    calories|protein| fat|sodium|
+---------+------------+-------+----+------+
| Cucumber| and Lemon "|   3.75|NULL|  NULL|
+---------+------------+-------+----+------+



In [32]:
for column in ["rating", "calories"]:
    food = food.where(is_a_number(F.col(column)))
    #la va a reemplazar porq ese nombre ya existe!
    food = food.withColumn(column, F.col(column).cast(T.DoubleType()))

food.count()

20048

In [33]:
food.select(CONTINUOUS).summary(
    "mean",
 "stddev",
 "min",
 "1%",
 "5%",
 "50%",
 "95%",
 "99%",
 "max",
).show()

+-------+------------------+------------------+------------------+-----------------+------------------+
|summary|            rating|          calories|           protein|              fat|            sodium|
+-------+------------------+------------------+------------------+-----------------+------------------+
|   mean| 3.714460295291301|6324.0634571930705|100.17385283565179|346.9398083953107| 6226.927244193346|
| stddev|1.3409187660508957|359079.83696340176| 3840.680997128738|20458.04034412412|333349.56803702697|
|    min|               0.0|               0.0|               0.0|              0.0|               0.0|
|     1%|               0.0|              18.0|               0.0|              0.0|               1.0|
|     5%|               0.0|              62.0|               0.0|              0.0|               5.0|
|    50%|             4.375|             331.0|               8.0|             17.0|             294.0|
|    95%|               5.0|            1315.0|              75.

In [34]:
20049

20049

In [37]:
maximum = {
    "calories":3184.0,
    "protein":171.0,
    "fat":205.0,
    "sodium":5649.0,
}


for k,v in maximum.items():
    food = food.withColumn(k, 
                    F.when(
                        F.isnull(F.col(k)), F.col(k)).
                        otherwise(
                            F.least(F.col(k), F.lit(v))
                        ),
                   )

food.where("calories > 3184.0").select("calories").show()

+--------+
|calories|
+--------+
+--------+

